**REPRESENTACIÓN BINARIA**

In [2]:
import random
import numpy as np
import pandas as pd

df = pd.read_csv('../notas_1u.csv')
alumnos = df['Alumno'].tolist()
notas = df['Nota'].tolist()

def crear_cromosoma():
    cromosoma = []
    for i in range(39):
        examen = random.randint(0, 2)
        genes = [0, 0, 0]
        genes[examen] = 1
        cromosoma.extend(genes)
    return cromosoma

def decodificar_cromosoma(cromosoma):
    asignaciones = {'A': [], 'B': [], 'C': []}
    examenes = ['A', 'B', 'C']
    
    for i in range(39):
        idx = i * 3
        for j in range(3):
            if cromosoma[idx + j] == 1:
                asignaciones[examenes[j]].append(i)
                break
    
    return asignaciones

def calcular_fitness(cromosoma):
    asignaciones = decodificar_cromosoma(cromosoma)
    
    if any(len(asignaciones[ex]) != 13 for ex in ['A', 'B', 'C']):
        return -1000
    
    promedios = {}
    for examen in ['A', 'B', 'C']:
        indices = asignaciones[examen]
        notas_examen = [notas[i] for i in indices]
        promedios[examen] = np.mean(notas_examen)
    
    desviacion = np.std(list(promedios.values()))
    return -desviacion

def mutacion(cromosoma):
    cromosoma_mutado = cromosoma.copy()
    
    alumno1 = random.randint(0, 38)
    alumno2 = random.randint(0, 38)
    
    idx1 = alumno1 * 3
    idx2 = alumno2 * 3
    
    examen1 = [i for i in range(3) if cromosoma_mutado[idx1 + i] == 1][0]
    examen2 = [i for i in range(3) if cromosoma_mutado[idx2 + i] == 1][0]
    
    if examen1 != examen2:
        cromosoma_mutado[idx1:idx1+3] = [0, 0, 0]
        cromosoma_mutado[idx1 + examen2] = 1
        
        cromosoma_mutado[idx2:idx2+3] = [0, 0, 0]
        cromosoma_mutado[idx2 + examen1] = 1
    
    return cromosoma_mutado

def algoritmo_genetico(generaciones=100, tam_poblacion=50):
    poblacion = [crear_cromosoma() for _ in range(tam_poblacion)]
    
    for gen in range(generaciones):
        fitness_scores = [(crom, calcular_fitness(crom)) for crom in poblacion]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        
        nueva_poblacion = []
        
        elite = int(tam_poblacion * 0.2)
        for i in range(elite):
            nueva_poblacion.append(fitness_scores[i][0])
        
        while len(nueva_poblacion) < tam_poblacion:
            padre = random.choice(poblacion[:tam_poblacion//2])
            hijo = mutacion(padre)
            nueva_poblacion.append(hijo)
        
        poblacion = nueva_poblacion
        
        if gen % 20 == 0:
            mejor_fitness = fitness_scores[0][1]
            print(f"Generación {gen}: Mejor fitness = {mejor_fitness:.4f}")
    
    mejor_cromosoma = fitness_scores[0][0]
    return mejor_cromosoma

print("REPRESENTACIÓN BINARIA")
print("Problema: Distribuir 39 alumnos en 3 exámenes (A, B, C) de forma equitativa")
print("Cromosoma: 117 bits (39 alumnos × 3 bits cada uno)")
print("Gen: [0,1,0] significa alumno asignado a examen B\n")

mejor_solucion = algoritmo_genetico()
asignaciones_finales = decodificar_cromosoma(mejor_solucion)

print("\nDistribución final:")
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedio = np.mean(notas_examen)
    print(f"Examen {examen}: {len(indices)} alumnos, promedio = {promedio:.2f}")
    print(f"  Alumnos: {[alumnos[i] for i in indices[:5]]}... (mostrando primeros 5)")

print("\nVerificación de equilibrio:")
promedios = []
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedios.append(np.mean(notas_examen))
print(f"Desviación estándar entre promedios: {np.std(promedios):.4f}")

REPRESENTACIÓN BINARIA
Problema: Distribuir 39 alumnos en 3 exámenes (A, B, C) de forma equitativa
Cromosoma: 117 bits (39 alumnos × 3 bits cada uno)
Gen: [0,1,0] significa alumno asignado a examen B

Generación 0: Mejor fitness = -1000.0000
Generación 20: Mejor fitness = -1000.0000
Generación 40: Mejor fitness = -1000.0000
Generación 60: Mejor fitness = -1000.0000
Generación 80: Mejor fitness = -1000.0000

Distribución final:
Examen A: 10 alumnos, promedio = 15.00
  Alumnos: ['Alumno1', 'Alumno4', 'Alumno8', 'Alumno11', 'Alumno14']... (mostrando primeros 5)
Examen B: 19 alumnos, promedio = 15.47
  Alumnos: ['Alumno2', 'Alumno6', 'Alumno7', 'Alumno9', 'Alumno13']... (mostrando primeros 5)
Examen C: 10 alumnos, promedio = 15.70
  Alumnos: ['Alumno3', 'Alumno5', 'Alumno10', 'Alumno12', 'Alumno15']... (mostrando primeros 5)

Verificación de equilibrio:
Desviación estándar entre promedios: 0.2917


**REPRESENTACIÓN PERMUTACIONAL**

In [3]:
import random
import numpy as np
import pandas as pd

df = pd.read_csv('../notas_1u.csv')
alumnos = df['Alumno'].tolist()
notas = df['Nota'].tolist()

def crear_cromosoma():
    indices = list(range(39))
    random.shuffle(indices)
    return indices

def decodificar_cromosoma(cromosoma):
    asignaciones = {
        'A': cromosoma[0:13],
        'B': cromosoma[13:26],
        'C': cromosoma[26:39]
    }
    return asignaciones

def calcular_fitness(cromosoma):
    asignaciones = decodificar_cromosoma(cromosoma)
    
    promedios = {}
    for examen in ['A', 'B', 'C']:
        indices = asignaciones[examen]
        notas_examen = [notas[i] for i in indices]
        promedios[examen] = np.mean(notas_examen)
    
    desv_promedios = np.std(list(promedios.values()))
    
    bonus_diversidad = 0
    for examen in ['A', 'B', 'C']:
        indices = asignaciones[examen]
        notas_examen = [notas[i] for i in indices]
        if max(notas_examen) - min(notas_examen) > 5:
            bonus_diversidad += 0.1
    
    fitness = -desv_promedios + bonus_diversidad
    return fitness

def cruce_pmx(padre1, padre2):
    size = len(padre1)
    punto1 = random.randint(0, size - 2)
    punto2 = random.randint(punto1 + 1, size)
    
    hijo = [-1] * size
    hijo[punto1:punto2] = padre1[punto1:punto2]
    
    mapeo = {}
    for i in range(punto1, punto2):
        mapeo[padre2[i]] = padre1[i]
    
    for i in range(size):
        if hijo[i] == -1:
            valor = padre2[i]
            while valor in hijo[punto1:punto2]:
                valor = mapeo.get(valor, valor)
            hijo[i] = valor
    
    return hijo

def mutacion_intercambio(cromosoma):
    cromosoma_mutado = cromosoma.copy()
    
    if random.random() < 0.3:
        pos1 = random.randint(0, 38)
        pos2 = random.randint(0, 38)
        
        cromosoma_mutado[pos1], cromosoma_mutado[pos2] = cromosoma_mutado[pos2], cromosoma_mutado[pos1]
    
    return cromosoma_mutado

def mutacion_inversion(cromosoma):
    cromosoma_mutado = cromosoma.copy()
    
    if random.random() < 0.2:
        inicio = random.randint(0, 36)
        longitud = random.randint(2, min(5, 39 - inicio))
        segmento = cromosoma_mutado[inicio:inicio + longitud]
        segmento.reverse()
        cromosoma_mutado[inicio:inicio + longitud] = segmento
    
    return cromosoma_mutado

def algoritmo_genetico(generaciones=50, tam_poblacion=30):
    poblacion = [crear_cromosoma() for _ in range(tam_poblacion)]
    
    historial_fitness = []
    
    for gen in range(generaciones):
        fitness_scores = [(crom, calcular_fitness(crom)) for crom in poblacion]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        
        historial_fitness.append(fitness_scores[0][1])
        
        nueva_poblacion = []
        
        elite = int(tam_poblacion * 0.2)
        for i in range(elite):
            nueva_poblacion.append(fitness_scores[i][0])
        
        while len(nueva_poblacion) < tam_poblacion:
            padre1 = random.choice(poblacion[:tam_poblacion//3])
            padre2 = random.choice(poblacion[:tam_poblacion//3])
            
            hijo = mutacion_intercambio(padre1)
            hijo = mutacion_inversion(hijo)
            
            nueva_poblacion.append(hijo)
        
        poblacion = nueva_poblacion
        
        if gen % 10 == 0:
            print(f"Generación {gen}: Mejor fitness = {fitness_scores[0][1]:.4f}")
    
    mejor_cromosoma = fitness_scores[0][0]
    return mejor_cromosoma, historial_fitness

print("REPRESENTACIÓN PERMUTACIONAL")
print("Problema: Secuenciar alumnos para asignación ordenada a exámenes")
print("Cromosoma: Permutación de 39 índices de alumnos")
print("Decodificación: Posiciones [0-12] → Examen A, [13-25] → Examen B, [26-38] → Examen C\n")

mejor_solucion, historial = algoritmo_genetico()
asignaciones_finales = decodificar_cromosoma(mejor_solucion)

print("\nAsignación final por orden de secuencia:")
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedio = np.mean(notas_examen)
    print(f"\nExamen {examen}: {len(indices)} alumnos, promedio = {promedio:.2f}")
    print(f"  Secuencia de alumnos:")
    for i, idx in enumerate(indices):
        print(f"    Posición {i+1}: {alumnos[idx]} (Nota: {notas[idx]})")
        if i >= 4:
            print("    ... (mostrando primeros 5)")
            break

print("\nEstadísticas finales:")
promedios = []
rangos = []
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedios.append(np.mean(notas_examen))
    rangos.append(max(notas_examen) - min(notas_examen))

print(f"Promedios: A={promedios[0]:.2f}, B={promedios[1]:.2f}, C={promedios[2]:.2f}")
print(f"Rangos de notas: A={rangos[0]:.0f}, B={rangos[1]:.0f}, C={rangos[2]:.0f}")
print(f"Desviación estándar entre promedios: {np.std(promedios):.4f}")

print("\nEvolución del algoritmo:")
print(f"Fitness inicial: {historial[0]:.4f}")
print(f"Fitness final: {historial[-1]:.4f}")
print(f"Mejora total: {((historial[-1] - historial[0]) / abs(historial[0]) * 100):.1f}%")


#

REPRESENTACIÓN PERMUTACIONAL
Problema: Secuenciar alumnos para asignación ordenada a exámenes
Cromosoma: Permutación de 39 índices de alumnos
Decodificación: Posiciones [0-12] → Examen A, [13-25] → Examen B, [26-38] → Examen C

Generación 0: Mejor fitness = 0.1187
Generación 10: Mejor fitness = 0.2041
Generación 20: Mejor fitness = 0.2637
Generación 30: Mejor fitness = 0.2637
Generación 40: Mejor fitness = 0.2637

Asignación final por orden de secuencia:

Examen A: 13 alumnos, promedio = 15.38
  Secuencia de alumnos:
    Posición 1: Alumno24 (Nota: 11.0)
    Posición 2: Alumno22 (Nota: 20.0)
    Posición 3: Alumno15 (Nota: 19.0)
    Posición 4: Alumno27 (Nota: 18.0)
    Posición 5: Alumno14 (Nota: 17.0)
    ... (mostrando primeros 5)

Examen B: 13 alumnos, promedio = 15.46
  Secuencia de alumnos:
    Posición 1: Alumno9 (Nota: 13.0)
    Posición 2: Alumno36 (Nota: 14.0)
    Posición 3: Alumno29 (Nota: 14.0)
    Posición 4: Alumno2 (Nota: 13.0)
    Posición 5: Alumno32 (Nota: 18.0)
    

**REPRESENTACIÓN REAL**

In [4]:
import random
import numpy as np
import pandas as pd

df = pd.read_csv('../notas_1u.csv')
alumnos = df['Alumno'].tolist()
notas = df['Nota'].tolist()

def crear_cromosoma():
    cromosoma = []
    for i in range(39):
        pesos = [random.random() for _ in range(3)]
        suma = sum(pesos)
        pesos_norm = [p/suma for p in pesos]
        cromosoma.extend(pesos_norm)
    return cromosoma

def decodificar_cromosoma(cromosoma):
    asignaciones = {'A': [], 'B': [], 'C': []}
    examenes = ['A', 'B', 'C']
    
    alumnos_disponibles = list(range(39))
    contadores = {'A': 0, 'B': 0, 'C': 0}
    
    while alumnos_disponibles:
        mejor_alumno = None
        mejor_examen = None
        mejor_valor = -1
        
        for alumno in alumnos_disponibles:
            idx = alumno * 3
            for i, examen in enumerate(examenes):
                if contadores[examen] < 13:
                    valor = cromosoma[idx + i]
                    if valor > mejor_valor:
                        mejor_valor = valor
                        mejor_alumno = alumno
                        mejor_examen = examen
        
        if mejor_alumno is not None:
            asignaciones[mejor_examen].append(mejor_alumno)
            contadores[mejor_examen] += 1
            alumnos_disponibles.remove(mejor_alumno)
    
    return asignaciones

def calcular_fitness(cromosoma):
    asignaciones = decodificar_cromosoma(cromosoma)
    
    promedios = {}
    varianzas = {}
    
    for examen in ['A', 'B', 'C']:
        indices = asignaciones[examen]
        notas_examen = [notas[i] for i in indices]
        promedios[examen] = np.mean(notas_examen)
        varianzas[examen] = np.var(notas_examen)
    
    desv_promedios = np.std(list(promedios.values()))
    promedio_varianzas = np.mean(list(varianzas.values()))
    
    fitness = -desv_promedios - 0.1 * promedio_varianzas
    return fitness

def cruce(padre1, padre2):
    hijo = []
    for i in range(39):
        idx = i * 3
        if random.random() < 0.5:
            genes = padre1[idx:idx+3]
        else:
            genes = padre2[idx:idx+3]
        
        genes = [g + random.gauss(0, 0.1) for g in genes]
        genes = [max(0, g) for g in genes]
        suma = sum(genes)
        if suma > 0:
            genes = [g/suma for g in genes]
        else:
            genes = [1/3, 1/3, 1/3]
        
        hijo.extend(genes)
    
    return hijo

def mutacion(cromosoma):
    cromosoma_mutado = cromosoma.copy()
    
    for i in range(39):
        if random.random() < 0.1:
            idx = i * 3
            nuevos_pesos = [random.random() for _ in range(3)]
            suma = sum(nuevos_pesos)
            cromosoma_mutado[idx:idx+3] = [p/suma for p in nuevos_pesos]
    
    return cromosoma_mutado

def algoritmo_genetico(generaciones=150, tam_poblacion=100):
    poblacion = [crear_cromosoma() for _ in range(tam_poblacion)]
    
    mejor_global_fitness = float('-inf')
    mejor_global_cromosoma = None
    
    for gen in range(generaciones):
        fitness_scores = [(crom, calcular_fitness(crom)) for crom in poblacion]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        
        if fitness_scores[0][1] > mejor_global_fitness:
            mejor_global_fitness = fitness_scores[0][1]
            mejor_global_cromosoma = fitness_scores[0][0].copy()
        
        nueva_poblacion = []
        
        elite = int(tam_poblacion * 0.1)
        for i in range(elite):
            nueva_poblacion.append(fitness_scores[i][0])
        
        while len(nueva_poblacion) < tam_poblacion:
            padre1 = random.choice(poblacion[:tam_poblacion//4])[0] if isinstance(poblacion[0], tuple) else random.choice(poblacion[:tam_poblacion//4])
            padre2 = random.choice(poblacion[:tam_poblacion//4])[0] if isinstance(poblacion[0], tuple) else random.choice(poblacion[:tam_poblacion//4])
            
            hijo = cruce(padre1, padre2)
            hijo = mutacion(hijo)
            nueva_poblacion.append(hijo)
        
        poblacion = nueva_poblacion
        
        if gen % 30 == 0:
            print(f"Generación {gen}: Mejor fitness = {fitness_scores[0][1]:.4f}")
    
    return mejor_global_cromosoma

print("REPRESENTACIÓN REAL")
print("Problema: Optimizar distribución de alumnos usando pesos probabilísticos")
print("Cromosoma: 117 valores reales (39 alumnos × 3 pesos normalizados)")
print("Gen: [0.2, 0.5, 0.3] representa probabilidades para exámenes A, B, C\n")

mejor_solucion = algoritmo_genetico()
asignaciones_finales = decodificar_cromosoma(mejor_solucion)

print("\nDistribución optimizada:")
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedio = np.mean(notas_examen)
    varianza = np.var(notas_examen)
    print(f"Examen {examen}: {len(indices)} alumnos")
    print(f"  Promedio: {promedio:.2f}, Varianza: {varianza:.2f}")
    print(f"  Rango de notas: [{min(notas_examen):.0f} - {max(notas_examen):.0f}]")

print("\nAnálisis de equilibrio:")
promedios = []
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedios.append(np.mean(notas_examen))

print(f"Promedios por examen: A={promedios[0]:.2f}, B={promedios[1]:.2f}, C={promedios[2]:.2f}")
print(f"Desviación estándar entre promedios: {np.std(promedios):.4f}")
print(f"Diferencia máxima entre promedios: {max(promedios) - min(promedios):.2f}")

REPRESENTACIÓN REAL
Problema: Optimizar distribución de alumnos usando pesos probabilísticos
Cromosoma: 117 valores reales (39 alumnos × 3 pesos normalizados)
Gen: [0.2, 0.5, 0.3] representa probabilidades para exámenes A, B, C

Generación 0: Mejor fitness = -1.0911
Generación 30: Mejor fitness = -1.0911
Generación 60: Mejor fitness = -1.0911
Generación 90: Mejor fitness = -1.0911
Generación 120: Mejor fitness = -1.0911

Distribución optimizada:
Examen A: 13 alumnos
  Promedio: 15.38, Varianza: 13.01
  Rango de notas: [9 - 20]
Examen B: 13 alumnos
  Promedio: 15.46, Varianza: 10.09
  Rango de notas: [9 - 20]
Examen C: 13 alumnos
  Promedio: 15.38, Varianza: 8.54
  Rango de notas: [11 - 20]

Análisis de equilibrio:
Promedios por examen: A=15.38, B=15.46, C=15.38
Desviación estándar entre promedios: 0.0363
Diferencia máxima entre promedios: 0.08
